In [1]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv("C:/Users/nkm/Downloads/Nat_Gas (1).csv")

In [12]:
df["Dates"] = pd.to_datetime(df["Dates"])
df = df.sort_values("Dates")

In [7]:
print(df.columns)


Index(['Dates', 'Prices'], dtype='object')


In [13]:
df.head()

,Dates,Prices,Date
0,2020-10-31,10.1,2020-10-31
1,2020-11-30,10.3,2020-11-30
2,2020-12-31,11.0,2020-12-31
3,2021-01-31,10.9,2021-01-31
4,2021-02-28,10.9,2021-02-28


In [14]:
def estimate_gas_price(input_date):
    import numpy as np
    import pandas as pd

    input_date = pd.to_datetime(input_date)

    # Convert your date column into numbers for regression
    df["timestamp"] = df["Dates"].map(pd.Timestamp.toordinal)

    # Fit linear trend line
    coeffs = np.polyfit(df["timestamp"], df["Prices"], 1)

    # Convert input date
    input_timestamp = input_date.toordinal()

    # Estimate price
    estimated_price = coeffs[0] * input_timestamp + coeffs[1]

    return round(float(estimated_price), 2)


In [15]:
estimate_gas_price("2026-12-31")


13.18

In [16]:
def price_storage_contract(
    injection_dates,
    withdrawal_dates,
    price_function,
    max_volume,
    rate_limit,
    storage_cost_per_month
):
    import pandas as pd

    stored_volume = 0
    total_cost = 0
    total_revenue = 0

    events = []

    # Add all injection events
    for d in injection_dates:
        events.append((pd.to_datetime(d), "inject"))

    # Add all withdrawal events
    for d in withdrawal_dates:
        events.append((pd.to_datetime(d), "withdraw"))

    # Sort events by date
    events.sort()
    last_date = events[0][0]

    for date, action in events:
        # Calculate storage cost for time gap
        months_stored = (date.year - last_date.year) * 12 + (date.month - last_date.month)
        total_cost += stored_volume * months_stored * storage_cost_per_month

        # Get estimated market price
        price = price_function(date)

        if action == "inject":
            volume = min(rate_limit, max_volume - stored_volume)
            total_cost += volume * price
            stored_volume += volume

        elif action == "withdraw":
            volume = min(rate_limit, stored_volume)
            total_revenue += volume * price
            stored_volume -= volume

        last_date = date

    return round(total_revenue - total_cost, 2)


In [17]:
# Example contract
injections = ["2024-01-01", "2024-02-01"]
withdrawals = ["2024-11-01", "2024-12-01"]

profit = price_storage_contract(
    injection_dates=injections,
    withdrawal_dates=withdrawals,
    price_function=estimate_gas_price,
    max_volume=1000,
    rate_limit=500,
    storage_cost_per_month=0.10
)

print("Estimated Contract Profit:", profit)


Estimated Contract Profit: -605.0


In [ ]:
# “With these buy/sell dates, storage costs + market prices caused a LOSS.”

In [18]:
# Try a Better Strategy (Make It Profitable)
# Smarter contract strategy
injections = ["2024-03-01", "2024-04-01"]   # Spring (cheaper gas)
withdrawals = ["2024-11-01", "2024-12-01"] # Winter (expensive gas)

profit = price_storage_contract(
    injection_dates=injections,
    withdrawal_dates=withdrawals,
    price_function=estimate_gas_price,
    max_volume=1000,
    rate_limit=500,
    storage_cost_per_month=0.05
)

print("Estimated Contract Profit:", profit)


Estimated Contract Profit: -85.0


In [ ]:
# This is actually great, Trisha 
#A small loss (-85) means your strategy almost worked

In [ ]:
# README.FILE